In [3]:
from PIL import Image
import imageio
import os
import glob
import math

import re

def sort_scenes(l):
    convert = lambda text: float(text) if text.isdigit() else text
    alphanum = lambda key: [convert(c) for c in re.split('([-+]?[0-9]*\.?[0-9]*)', key)]
    l.sort(key=alphanum)
    return l

In [4]:
base_path = '/mnt/hdd/mvsnerf_data/dtu/Rectified'
scenes = sort_scenes(os.listdir(base_path))
scene_paths = [os.path.join(base_path, scene) for scene in scenes]

camera = 47
light = 5
image_name = f'rect_{camera:03d}_{light}_r5000.png'

image_paths = [os.path.join(scene_path, image_name) for scene_path in scene_paths]

In [5]:
num_images = len(image_paths)
num_sqrt = math.sqrt(num_images)

if num_sqrt.is_integer():
    tile_height = num_sqrt
    tile_width = num_sqrt
    tile_fill = 0
    
else:
    num_ceil = math.ceil(num_sqrt)
    num_floor = math.floor(num_sqrt)
    
    ceil_mul_floor = num_ceil * num_floor
    
    if num_images <= (ceil_mul_floor):
        tile_width = num_ceil
        tile_height = num_floor
        tile_fill = ceil_mul_floor - num_images
        
    if num_images > (ceil_mul_floor):
        tile_height = num_ceil
        tile_width = num_ceil
        tile_fill = num_ceil * num_ceil - num_images        

print(tile_width, tile_height, tile_fill)

11 11 2


In [6]:
for i, image_path in enumerate(image_paths):
    image = Image.open(image_path)
    
    if i == 0:
        image_width = image.width
        image_height = image.height
        
        dst = Image.new('RGB', (image.width * tile_width, image.height * tile_height))
    
    dst.paste(image, (image_width * (i % tile_width), image_height * (i // tile_width)))

In [7]:
dst.save(f'./all_dtu_scenes_{camera:03d}_{light}_r5000.png')

In [21]:
# make as function

def get_combined_image_dimension(image_paths):
    num_images = len(image_paths)
    num_sqrt = math.sqrt(num_images)

    if num_sqrt.is_integer():
        tile_height = num_sqrt
        tile_width = num_sqrt
        tile_fill = 0

    else:
        num_ceil = math.ceil(num_sqrt)
        num_floor = math.floor(num_sqrt)

        ceil_mul_floor = num_ceil * num_floor

        if num_images <= (ceil_mul_floor):
            tile_width = num_ceil
            tile_height = num_floor
            tile_fill = ceil_mul_floor - num_images

        if num_images > (ceil_mul_floor):
            tile_width = num_ceil
            tile_height = num_ceil
            tile_fill = num_ceil * num_ceil - num_images        
    
    return tile_width, tile_height, tile_fill


def combine_all_scenes(base_path, camera, light):
    
    scenes = sort_scenes(os.listdir(base_path))
    scene_paths = [os.path.join(base_path, scene) for scene in scenes]
    
    image_name = f'rect_{camera:03d}_{light}_r5000.png'
    image_paths = [os.path.join(scene_path, image_name) for scene_path in scene_paths]
    
    tile_width, tile_height, _ = get_combined_image_dimension(image_paths)
    
    for i, image_path in enumerate(image_paths):
        image = Image.open(image_path)

        if i == 0:
            image_width = image.width
            image_height = image.height

            dst = Image.new('RGB', (image.width * tile_width, image.height * tile_height))

        dst.paste(image, (image_width * (i % tile_width), image_height * (i // tile_width)))
    
    save_dir = f'/mnt/hdd/mvsnerf_data/all_dtu_scenes/{light}'
    save_name = f'all_dtu_scenes_{camera:03d}_{light}_r5000.png'
    
    os.makedirs(save_dir, exist_ok=True)
    
    dst.save(os.path.join(save_dir, save_name))
    
def create_video(save_dir, light, fps):
    file_list = glob.glob(os.path.join(save_dir, '*.png'))
    file_list.sort()
    
    os.makedirs(os.path.join(save_dir, 'video'), exist_ok=True)
    save_video_path = os.path.join(save_dir, f'video/{light}.mp4')
    
    writer = imageio.get_writer(save_video_path, fps=fps)

    for im in file_list:
        writer.append_data(imageio.imread(im))
    writer.close()

In [15]:
base_path = '/mnt/hdd/mvsnerf_data/dtu/Rectified'
light = 6

for camera in range(1, 48+1):   
    combine_all_scenes(base_path, camera, light)

In [ ]:
light = 1

In [16]:
save_dir = f'/mnt/hdd/mvsnerf_data/all_dtu_scenes/{light}/'

file_list = glob.glob(os.path.join(save_dir, '*.png'))
file_list.sort()

In [17]:
os.makedirs(os.path.join(save_dir, 'video'), exist_ok=True)

save_video_path = os.path.join(save_dir, f'video/{light}.mp4')

In [19]:
writer = imageio.get_writer(save_video_path, fps=6)

for im in file_list:
    writer.append_data(imageio.imread(im))
writer.close()

In [22]:
save_dir = f'/mnt/hdd/mvsnerf_data/all_dtu_scenes/{light}/'
for light in [1, 6]:
    create_video(save_dir, light, fps=12)